In [245]:
# analisis de dades
import pandas as pd #processament de dades
import numpy as np #algebra lineal
import random as rnd

# visualització
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

## Agafem les dades
Un dataset per caga grup de dàdes i un conjunt

In [484]:
train_data = pd.read_csv('./input/train.csv')
test_data = pd.read_csv('./input/test.csv')
combine_data = [train_data, test_data]

## Quins camps tenen les dàdes?
**que volen dir aquets camps?**


In [485]:
print(train_data.columns.values)

['PassengerId' 'Survived' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch'
 'Ticket' 'Fare' 'Cabin' 'Embarked']


- Survival - Van sobreviure? (0 = No; 1 = Si). Al test.csv no hi ha aquest camp (s'ha de predir).
- Pclass - La classe del passatger (1 = alta ; 2 = mitja; 3 = baixa)
- Name - Nom
- Sex - Sexe
- Age - Edat
- Sibsp - Nombre de germans / espòs-esposa a bord
- Parch - Nombre de pares / fills a bord
- Ticket - Numero del tiquet
- Fare - Tarifa del tiquet comprat pel passatger
- Cabin - Número de cabina 
- Embarked - Port desde on van embarcar(C = Cherbourg; Q = Queenstown; S = Southampton)

In [486]:
#mirem les dades per dalt
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [487]:
#mirem les dades per abaix
#veiem que hi ha valors nulls
train_data.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [488]:
train_data.info()
print('_'*40)
test_data.info()
#podem veure com hi ha camps que tenen valors nulls

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Passenger

## Analitzant les dades segons categories

In [489]:
#ha sobreviscut segons la classe
train_data[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [490]:
#ha sobreviscut segons el sexe
train_data[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [491]:
#ha sobreviscut segons la classe
train_data[["SibSp", "Survived"]].groupby(['SibSp'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,SibSp,Survived
1,1,0.535885
2,2,0.464286
0,0,0.345395
3,3,0.250000
4,4,0.166667
5,5,0.000000
6,8,0.000000


In [492]:
#ha sobreviscut segons Nombre de germans / espòs-esposa a bord
train_data[["Parch", "Survived"]].groupby(['Parch'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Parch,Survived
3,3,0.600000
1,1,0.550847
2,2,0.500000
0,0,0.343658
5,5,0.200000
4,4,0.000000
6,6,0.000000


In [493]:
#ha sobreviscut segons grups d'edat
age = pd.cut(train_data["Age"], bins=[0, 10, 20, 30, 40, 50, 60, 70, np.inf])
train_data.groupby(age).agg({'Survived' : 'mean'}).sort_values(by='Survived', ascending=False)

,Survived
Age,
"(0.0, 10.0]",0.593750
"(30.0, 40.0]",0.445161
"(50.0, 60.0]",0.404762
"(40.0, 50.0]",0.383721
"(10.0, 20.0]",0.382609
"(20.0, 30.0]",0.365217
"(60.0, 70.0]",0.235294
"(70.0, inf]",0.200000


## Tractem les dades

Tractarem les dades perqué siguin llegibles pel model matemàtic 

***Descartem les columnes Ticket, Cabin i Name***
- En el càs del train_data, El PassengerId no ens aporta res

In [494]:
train_data = train_data.drop(['Ticket', 'Cabin','Name','PassengerId'], axis=1)
test_data = test_data.drop(['Ticket', 'Cabin','Name'], axis=1)
combine_data = [train_data, test_data]

***Convertim el sex amb dades llegibles pel model matemàtic***


In [495]:
for dataset in combine_data:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_data.head()


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.0,1,0,7.2500,S
1,1,1,1,38.0,1,0,71.2833,C
2,1,3,1,26.0,0,0,7.9250,S
3,1,1,1,35.0,1,0,53.1000,S
4,0,3,0,35.0,0,0,8.0500,S


In [496]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,34.5,0,0,7.8292,Q
1,893,3,1,47.0,1,0,7.0000,S
2,894,2,0,62.0,0,0,9.6875,Q
3,895,3,0,27.0,0,0,8.6625,S
4,896,3,1,22.0,1,1,12.2875,S


***Convertim el paràmetre edat en dades llegibles pel model matemàtic***
- Agruparem les edats amb 8 grups:
  -  0-10 / 10-20 / 20-30 / 30-40 / 40-50 / 50-60 / 60-70 / 70-inf=80 


In [497]:
#resolem el tema de que hi han passatgers amb l'edat amb null
train_data.info()

train_data['Age']=train_data.groupby(['Pclass','Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
test_data['Age']=test_data.groupby(['Pclass','Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
combine_data = [train_data, test_data]
print('_'*40)

train_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       714 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Fare      891 non-null    float64
 7   Embarked  889 non-null    object 
dtypes: float64(2), int32(1), int64(4), object(1)
memory usage: 52.3+ KB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    int32  
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     

In [498]:
#passem a int el valor age
for dataset in combine_data:
    dataset['Age'] = dataset['Age'].astype(int)

In [499]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22,1,0,7.2500,S
1,1,1,1,38,1,0,71.2833,C
2,1,3,1,26,0,0,7.9250,S
3,1,1,1,35,1,0,53.1000,S
4,0,3,0,35,0,0,8.0500,S


In [500]:
#dividim les edats en 8 grups
train_data['AgeGroup'] = pd.cut(train_data['Age'], 8)
train_data[['AgeGroup', 'Survived']].groupby(['AgeGroup'], as_index=False).mean().sort_values(by='AgeGroup', ascending=True)

,AgeGroup,Survived
0,"(-0.08, 10.0]",0.593750
1,"(10.0, 20.0]",0.379310
2,"(20.0, 30.0]",0.322751
3,"(30.0, 40.0]",0.448649
4,"(40.0, 50.0]",0.392857
5,"(50.0, 60.0]",0.404762
6,"(60.0, 70.0]",0.222222
7,"(70.0, 80.0]",0.250000


In [501]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeGroup
0,0,3,0,22,1,0,7.2500,S,"(20.0, 30.0]"
1,1,1,1,38,1,0,71.2833,C,"(30.0, 40.0]"
2,1,3,1,26,0,0,7.9250,S,"(20.0, 30.0]"
3,1,1,1,35,1,0,53.1000,S,"(30.0, 40.0]"
4,0,3,0,35,0,0,8.0500,S,"(30.0, 40.0]"


In [502]:
#assignem un número al grup d'edat
for dataset in combine_data:
    dataset.loc[(dataset['Age'] >= 0) & (dataset['Age'] <= 80), 'Age'] //= 10
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,AgeGroup
0,0,3,0,2,1,0,7.2500,S,"(20.0, 30.0]"
1,1,1,1,3,1,0,71.2833,C,"(30.0, 40.0]"
2,1,3,1,2,0,0,7.9250,S,"(20.0, 30.0]"
3,1,1,1,3,1,0,53.1000,S,"(30.0, 40.0]"
4,0,3,0,3,0,0,8.0500,S,"(30.0, 40.0]"


In [503]:
#descartem la columna AgeGroup
train_data = train_data.drop(['AgeGroup'], axis=1)
combine_data = [train_data, test_data]
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,2,1,0,7.2500,S
1,1,1,1,3,1,0,71.2833,C
2,1,3,1,2,0,0,7.9250,S
3,1,1,1,3,1,0,53.1000,S
4,0,3,0,3,0,0,8.0500,S


***Convertim el paràmetre Embarked en dades llegibles pel model matemàtic***
- S = 0
- C = 1
- Q = 2

In [504]:
port_frequent = train_data.Embarked.dropna().mode()[0]
port_frequent

'S'

In [505]:
#en els valors nulls hi posem el port mès freqüent, en aquest cas serà: S =  Southampton
for dataset in combine_data:
    dataset['Embarked'] = dataset['Embarked'].fillna(port_frequent)
    
train_data[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

,Embarked,Survived
0,C,0.553571
1,Q,0.389610
2,S,0.339009


In [506]:
#posem S=0,C=1 i Q=2
for dataset in combine_data:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,2,1,0,7.2500,0
1,1,1,1,3,1,0,71.2833,1
2,1,3,1,2,0,0,7.9250,0
3,1,1,1,3,1,0,53.1000,0
4,0,3,0,3,0,0,8.0500,0


***Convertim el paràmetre Fare en dades llegibles pel model matemàtic***

In [507]:
#omplim els llocs buits amb la mitjana
test_data['Fare'].fillna(test_data['Fare'].dropna().median(), inplace=True)
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,0,3,0,0,7.8292,2
1,893,3,1,4,1,0,7.0000,0
2,894,2,0,6,0,0,9.6875,2
3,895,3,0,2,0,0,8.6625,0
4,896,3,1,2,1,1,12.2875,0


In [508]:
#dividim el fare en 4 grups 
train_data['FareBand'] = pd.qcut(train_data['Fare'], 4)
train_data[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

,FareBand,Survived
0,"(-0.001, 7.91]",0.197309
1,"(7.91, 14.454]",0.303571
2,"(14.454, 31.0]",0.454955
3,"(31.0, 512.329]",0.581081


In [509]:
#posem un valor a cada fareBand
for dataset in combine_data:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_data = train_data.drop(['FareBand'], axis=1)
combine_data = [train_data, test_data]
    


***Descartem les columnes SibSp i Parch ***


In [510]:
train_data = train_data.drop(['SibSp', 'Parch'], axis=1)
test_data = test_data.drop(['SibSp', 'Parch'], axis=1)
combine_data = [train_data, test_data]

In [511]:
train_data.head(10)

,Survived,Pclass,Sex,Age,Fare,Embarked
0,0,3,0,2,0,0
1,1,1,1,3,3,1
2,1,3,1,2,1,0
3,1,1,1,3,3,0
4,0,3,0,3,1,0
5,0,3,0,2,1,2
6,0,1,0,5,3,0
7,0,3,0,0,2,0
8,1,3,1,2,1,0
9,1,2,1,1,2,1


In [512]:
test_data.head(10)

,PassengerId,Pclass,Sex,Age,Fare,Embarked
0,892,3,0,3,0,2
1,893,3,1,4,0,0
2,894,2,0,6,1,2
3,895,3,0,2,1,0
4,896,3,1,2,1,0
5,897,3,0,1,1,0
6,898,3,1,3,0,2
7,899,2,0,2,2,0
8,900,3,1,1,0,1
9,901,3,0,2,2,0


## Model matemàtic + predicció + resolució

- En el nostre càs utilitzarem el model KNN o k-Nearest Neighbors.
- N'hi han molts i cadescun respon diferent a les dades.
- No n'hi han de millors i pitjors.



In [513]:
X_train = train_data.drop("Survived", axis=1)
Y_train = train_data["Survived"]
X_test  = test_data.drop("PassengerId", axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

((891, 5), (891,), (418, 5))

## k-Nearest Neighbors algorithm 
In pattern recognition k-NN is a non-parametric method used for classification and regression. 

KNN confidence score is better than Logistics Regression but worse than SVM.

In [515]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

83.5

In [517]:
submission = pd.DataFrame({
        "PassengerId": test_data["PassengerId"],
        "Survived": Y_pred
    })
submission.to_csv('./output/submission.csv', index=False)

In [518]:
Y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,